In [1]:
import rasterio
from rasterio.enums import Resampling
import numpy as np
import os


In [6]:

def downsample_to_common_resolution(file_paths, output_dir):
    resolutions = []
    os.makedirs(output_dir, exist_ok=True)

    # Read resolutions of all the tif files
    for file_path in file_paths:
        with rasterio.open(file_path) as src:
            resolutions.append(src.res)

    # Find the common resolution (the largest resolution in each dimension)
    max_resolution = (max(res[0] for res in resolutions), max(res[1] for res in resolutions))

    # Process each tif file
    for file_path in file_paths:
        with rasterio.open(file_path) as src:
            if src.res != max_resolution:
                scale_x = src.res[0] / max_resolution[0]
                scale_y = src.res[1] / max_resolution[1]

                # Calculate the new dimensions
                new_width = int(src.width * scale_x)
                new_height = int(src.height * scale_y)

                # Read the data and resample
                data = src.read(
                    out_shape=(src.count, new_height, new_width),
                    resampling=Resampling.bilinear
                )

                # Update the metadata
                transform = src.transform * src.transform.scale(
                    (src.width / data.shape[-1]),
                    (src.height / data.shape[-2])
                )

                out_meta = src.meta.copy()
                out_meta.update({
                    "height": new_height,
                    "width": new_width,
                    "transform": transform
                })
            else:
                # If the resolution matches, no downsampling needed, use original data
                data = src.read()
                out_meta = src.meta.copy()

            # Write the data to a new file in the output directory
            output_path = os.path.join(output_dir, os.path.basename(file_path))
            with rasterio.open(output_path, "w", **out_meta) as dest:
                dest.write(data)


In [5]:
ground_truth_path = '../../gully_detection/raw_data/HUC_071100060307/GT/GT.tif'

stream_order_path = '../../gully_detection/raw_data/HUC_071100060307/merged/dem_tile__merged.tif'

input_files = [ground_truth_path,
               stream_order_path,
               '../../gully_detection/raw_data/HUC_071100060307/merged_tiffs/tile_10__merged.tif', 
                 '../../gully_detection/raw_data/HUC_071100060307/merged_tiffs/tile_12__merged.tif', 
                 '../../gully_detection/raw_data/HUC_071100060307/merged_tiffs/tile_14__merged.tif', 
                 '../../gully_detection/raw_data/HUC_071100060307/merged_tiffs/tile_16__merged.tif',
                 '../../gully_detection/raw_data/HUC_071100060307/merged_tiffs/tile_18__merged.tif', 
                 '../../gully_detection/raw_data/HUC_071100060307/merged_tiffs/tile_20__merged.tif']

output_dir = '../../gully_detection/res_test_1/'
# Directory to save the downsampled tif files
# output_directory = "path/to/output/directory"

downsample_to_common_resolution(input_files, output_dir)
